# Assignmen 5 vj222hx

## Conceptual
1. The data is split into K equally sized parts, usually 5 or 10. First, one part is used for testing and the remaining parts for training, then this is done for all the parts. Then the final result will be the average of the K tests. 

2. i. Validation set is faster to perform as kfold is essentially validation set approach but doing it k times however validation set is also less reliable and has a higher chance of overfitting.

2. ii. LOOCV is an exctreme case of K-Fold which is really computationally expensive since it is K-Fold where K=N. LOOCV however has a low chance of overfitting.

## Practical
Add imports

In [35]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

Load Auto dataset

In [36]:
csvFile = pd.read_csv("Auto.csv", index_col=0)

Display the number of features and their names:

In [37]:
colNames = csvFile.columns.tolist()
print(f"Number of columns: {len(colNames)}")
print(f"Column Names: {colNames}")

Number of columns: 9
Column Names: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']


Print a statistic summary of the predictors and the response:

In [38]:
print(csvFile.describe(), "\n")
print(csvFile['name'].value_counts())

              mpg   cylinders  displacement  horsepower       weight  \
count  392.000000  392.000000    392.000000  392.000000   392.000000   
mean    23.445918    5.471939    194.411990  104.469388  2977.584184   
std      7.805007    1.705783    104.644004   38.491160   849.402560   
min      9.000000    3.000000     68.000000   46.000000  1613.000000   
25%     17.000000    4.000000    105.000000   75.000000  2225.250000   
50%     22.750000    4.000000    151.000000   93.500000  2803.500000   
75%     29.000000    8.000000    275.750000  126.000000  3614.750000   
max     46.600000    8.000000    455.000000  230.000000  5140.000000   

       acceleration        year      origin  
count    392.000000  392.000000  392.000000  
mean      15.541327   75.979592    1.576531  
std        2.758864    3.683737    0.805518  
min        8.000000   70.000000    1.000000  
25%       13.775000   73.000000    1.000000  
50%       15.500000   76.000000    1.000000  
75%       17.025000   79.0000

Display the number of datapoints

In [39]:
print(f"Number of datapoints {len(csvFile)}")

Number of datapoints 392


Display the data in a table

In [40]:
print(csvFile.head(20))

     mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
1   18.0          8         307.0         130    3504          12.0    70   
2   15.0          8         350.0         165    3693          11.5    70   
3   18.0          8         318.0         150    3436          11.0    70   
4   16.0          8         304.0         150    3433          12.0    70   
5   17.0          8         302.0         140    3449          10.5    70   
6   15.0          8         429.0         198    4341          10.0    70   
7   14.0          8         454.0         220    4354           9.0    70   
8   14.0          8         440.0         215    4312           8.5    70   
9   14.0          8         455.0         225    4425          10.0    70   
10  15.0          8         390.0         190    3850           8.5    70   
11  15.0          8         383.0         170    3563          10.0    70   
12  14.0          8         340.0         160    3609           8.0    70   

Correlation Plot

In [41]:
sns.heatmap(csvFile.drop(columns=['name']).corr(), annot=True, fmt=".2f", linewidths=0.5)
plt.show()

Estimating accuracy of lin reg model

In [42]:
def boot_fn(data, index):
    sample = data.iloc[index]
    X = sample['horsepower']
    y = sample['mpg']
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model.params

In [43]:
print(boot_fn(csvFile, range(392)))

const         39.935861
horsepower    -0.157845
dtype: float64


In [44]:
np.random.seed(1)
print(boot_fn(csvFile, np.random.choice(392, 392, replace=True)))

const         39.658479
horsepower    -0.155898
dtype: float64


In [45]:
print(boot_fn(csvFile, np.random.choice(392, 392, replace=True)))

const         40.733271
horsepower    -0.163901
dtype: float64


In [46]:
boot_results = np.zeros((1000, 2))
for i in range(1000):
    indices = np.random.choice(392, 392, replace=True)  # Bootstrap resampling
    boot_results[i, :] = boot_fn(csvFile, indices)
print(f"standard errors: {boot_results.std(axis=0)}")


standard errors: [0.82541249 0.0071347 ]


In [47]:
X = csvFile['horsepower']
y = csvFile['mpg']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.params)
print(model.summary())

const         39.935861
horsepower    -0.157845
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Sun, 02 Mar 2025   Prob (F-statistic):           7.03e-81
Time:                        15:02:59   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

The standard errors in the statistics estimate are 0.717 and 0.006. The standard errors in the bootstrap are 0.825 and 0.007. Here we can see that the bootstrap estimate has a slightly higher standard error which is expected since it is an estimate over many different subsets, so the regular estimate is better according these numbers but it is probably because it is slightly overfitted. 

In [48]:
def boot_fn_quadratic(data, index):
    sample = data.iloc[index]
    X = sample['horsepower']
    X = np.column_stack((X, X**2))
    X = sm.add_constant(X)
    y = sample['mpg']
    model = sm.OLS(y, X).fit()
    return model.params

In [49]:
np.random.seed(1)
boot_results = np.zeros((1000, 3))
for i in range(1000):
    indices = np.random.choice(392, 392, replace=True)
    boot_results[i, :] = boot_fn_quadratic(csvFile, indices)
print(f"standard errors: {boot_results.std(axis=0)}")


standard errors: [2.10232492e+00 3.35021074e-02 1.21097674e-04]


In [50]:
csvFile['horsepower_squared'] = csvFile['horsepower'] ** 2
X = csvFile[['horsepower', 'horsepower_squared']]
y = csvFile['mpg']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.params)
print(model.summary())

const                 56.900100
horsepower            -0.466190
horsepower_squared     0.001231
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     428.0
Date:                Sun, 02 Mar 2025   Prob (F-statistic):           5.40e-99
Time:                        15:03:00   Log-Likelihood:                -1133.2
No. Observations:                 392   AIC:                             2272.
Df Residuals:                     389   BIC:                             2284.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------

Bootstrap standard errors: 2.10, 0.034, 0.00012

Statistical standard errors: 1.8, 0.031, 0.000

Here we can see that bootstrap once again has a higher standard error for the same reason as in the linear case. 